In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 1 # how many independent sequences will we process in parallel?
block_size = 3 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            print("shape of logits: ", logits.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)


        return logits


model = BigramLanguageModel(vocab_size)
m = model.to(device)

input = torch.tensor([[1,2,3,4,5,6,7,8]], dtype=torch.long, device=device)
out = m(input)

In [25]:
input.shape, out.shape

(torch.Size([1, 8]), torch.Size([1, 8, 65]))

In [23]:
input

tensor([[1, 2, 3, 4, 5, 6, 7, 8]])

In [45]:
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')


4.225 K parameters


In [46]:
[p.numel() for p in m.parameters()]

[4225]

In [32]:
out.view(8,65)

tensor([[ 5.9780e-01, -5.1406e-02, -6.4559e-02, -4.9701e-01,  4.6576e-01,
         -2.5726e-01, -1.0673e+00,  2.0089e+00, -5.3698e-01,  2.2280e-01,
          6.9705e-01, -1.4267e+00,  9.0594e-01,  1.4459e-01,  2.2800e-01,
          2.4900e+00, -1.2237e+00,  1.0107e+00,  5.5600e-01, -1.5935e+00,
         -1.2706e+00,  6.9033e-01, -1.9614e-01,  3.4491e-01, -3.4189e-01,
          4.7587e-01, -7.6634e-01, -4.1896e-01, -4.3699e-01, -1.0012e+00,
         -4.0943e-01, -1.6669e+00, -1.3651e+00, -1.6552e-01,  9.6225e-01,
          3.1549e-02, -7.4190e-01, -2.9779e-01,  1.7166e-02, -1.7722e-01,
         -1.3343e-01,  2.9396e-01,  1.3850e+00,  1.2091e-01,  2.5418e+00,
         -6.4046e-01, -1.9740e+00, -3.2957e-01,  7.9589e-03,  9.2623e-01,
         -1.8846e+00,  1.6696e-01,  4.5862e-01, -1.7662e+00,  5.8599e-01,
          1.7510e+00,  2.8072e-01,  3.1096e-01, -6.5376e-01, -6.5763e-01,
          3.1845e-01, -5.4959e-01, -1.4649e+00, -2.0555e+00,  1.8275e+00],
        [ 1.3035e+00, -4.5013e-01,  1

In [34]:
import torch
import torch.nn.functional as F

logits = torch.tensor([0.1, 0.2, 0.3, 0.4])
probabilities = F.softmax(logits, dim=0)

print(probabilities)

tensor([0.2138, 0.2363, 0.2612, 0.2887])


In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        B, T, C = logits.shape

        # tril = torch.tril(torch.ones(T, T))
        # wei = torch.zeros((T,T))
        # wei = wei.masked_fill(tril == 0, float('-inf'))
        # wei = F.softmax(wei, dim=-1)
        # logits = wei @ logits



        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911
step 3000: train loss 2.4613, val loss 2.4897
step 3300: train loss 2.4689, val loss 2.4793
step 3600: train loss 2.4554, val loss 2.4919
step 3900: train loss 2.4682, val loss 2.4906
step 4200: train loss 2.4634, val loss 2.4882
step 4500: train loss 2.4563, val loss 2.4804
step 4800: train loss 2.4557, val loss 2.4852
step 5100: train loss 2.4605, val loss 2.4926
step 5400: train loss 2.4614, val loss 2.4932
step 5700: train loss 2.4684, val loss 2.4781
step 6000: train loss 2.4651, val loss 2.4672
step 6300: train loss 2.4586, val loss 2